## Load data

In [1]:
from LoadSplitData import Process

Using TensorFlow backend.


In [2]:
data = Process()
data.split_data()

Images shape (24076, 80, 80, 3), label shape (24076,), ratio of authorized data 0.4503655092208008
X_train shape: (12639, 80, 80, 3)
12639 train samples
5418 valid samples
6019 test samples


## Train the model

In [3]:
from BuildModel import Model

In [4]:
model = Model()
model.build_model(data)
model.train(data)
model.save()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 80, 80, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 80, 80, 32)    9248        convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 40, 40, 32)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 40, 40, 32)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Evaluate the Model

In [5]:
model = Model()
model.load()
model.evaluate(data)

Model Loaded.
acc: 99.57%


## Predict new image

In [6]:
import cv2
import imutils
resized_height = 80

In [7]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized1.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[  4.80105655e-07   9.99999523e-01]]
1/1 [==============================] - 0s


1

In [8]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized2.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.00281041  0.99718958]]
1/1 [==============================] - 0s


1

In [9]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized3.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.9285748   0.07142527]]
1/1 [==============================] - 0s


0

In [10]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized4.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.00551697  0.99448305]]
1/1 [==============================] - 0s


1

In [11]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized5.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.99599791  0.00400205]]
1/1 [==============================] - 0s


0

In [12]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized6.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.92524755  0.07475249]]
1/1 [==============================] - 0s


0

In [13]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\authorized7.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.01909483  0.98090518]]
1/1 [==============================] - 0s


1

In [14]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\liyoubing.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.95637435  0.04362565]]
1/1 [==============================] - 0s


0

In [15]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhourunfa.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.02582469  0.97417533]]
1/1 [==============================] - 0s


1

In [16]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\chenbaoguo.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[  9.99981761e-01   1.82781805e-05]]
1/1 [==============================] - 0s


0

In [17]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhangguoli.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.96952581  0.03047414]]
1/1 [==============================] - 0s


0

In [18]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\lixuejian.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.99750501  0.00249495]]
1/1 [==============================] - 0s


0

In [19]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\wangziwen.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.99293858  0.00706143]]
1/1 [==============================] - 0s


0

In [20]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\tangguoqiang.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.08965806  0.91034192]]
1/1 [==============================] - 0s


1

In [21]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\zhangyimou.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.00535461  0.99464536]]
1/1 [==============================] - 0s


1

In [22]:
image = cv2.imread(r'C:\Users\dbsnail\ImageFolder\test\yimouzhang.jpg')
resized_image = imutils.resize(image, height = resized_height)
model.predict(resized_image)

1/1 [==============================] - 0s
[[ 0.14002162  0.85997832]]
1/1 [==============================] - 0s


1